# Multivariate SuSiE and ENLOC model

## Aim

This notebook aims to demonstrate a workflow of generating posterior inclusion probabilities (PIPs) from GWAS summary statistics using SuSiE regression and construsting SNP signal clusters from global eQTL analysis data obtained from multivariate SuSiE models.

## Methods overview

This procedure assumes that molecular phenotype summary statistics and GWAS summary statistics are aligned and harmonized to have consistent allele coding (see [this module](../../misc/summary_stats_merger.html) for implementation details). Both molecular phenotype QTL and GWAS should be fine-mapped beforehand using mvSusiE or SuSiE. We further assume (and require) that molecular phenotype and GWAS data come from the same population ancestry. Violations from this assumption may not cause an error in the analysis computational workflow but the results obtained may not be valid.

## Input

1) GWAS Summary Statistics with the following columns:
    - chr: chromosome number
    - bp: base pair position
    - a1: effect allele
    - a2: other allele
    - beta: effect size
    - se: standard error of beta
    - z: z score

2) eQTL data from multivariate SuSiE model with the following columns:
    - chr: chromosome number
    - bp: base pair position
    - a1: effect allele
    - a2: other allele
    - pip: posterior inclusion probability

3) LD correlation matrix

## Output

Intermediate files:

1) GWAS PIP file with the following columns
    - var_id
    - ld_block 
    - snp_pip
    - block_pip

2) eQTL annotation file with the following columns
    - chr
    - bp
    - var_id
    - a1
    - a2
    - annotations, in the format: `gene:cs_num@tissue=snp_pip[cs_pip:cs_total_snps]`


Final Outputs:

1) Enrichment analysis result prefix.enloc.enrich.rst: estimated enrichment parameters and standard errors.

2) Signal-level colocalization result prefix.enloc.sig.out: the main output from the colocalization analysis with the following format
    - column 1: signal cluster name (from eQTL analysis)
    - column 2: number of member SNPs
    - column 3: cluster PIP of eQTLs
    - column 4: cluster PIP of GWAS hits (without eQTL prior)
    - column 5: cluster PIP of GWAS hits (with eQTL prior)
    - column 6: regional colocalization probability (RCP)

3) SNP-level colocalization result prefix.enloc.snp.out: SNP-level colocalization output with the following form at
    - column 1: signal cluster name
    - column 2: SNP name
    - column 3: SNP-level PIP of eQTLs
    - column 4: SNP-level PIP of GWAS (without eQTL prior)
    - column 5: SNP-level PIP of GWAS (with eQTL prior)
    - column 6: SNP-level colocalization probability

4) Sorted list of colocalization signals

Takes into consideration 3 situations: 

1) "Major" and "minor" alleles flipped

2) Different strand but same variant

3) Remove variants with A/T and C/G alleles due to ambiguity

## Minimal working example

In [ ]:
sos run mvenloc.ipynb merge \
    --cwd output \
    --eqtl-sumstats .. \
    --gwas-sumstats ..

In [ ]:
sos run mvenloc.ipynb eqtl \
    --cwd output \
    --sumstats-file .. \
    --ld-region ..

In [ ]:
sos run mvenloc.ipynb gwas \
    --cwd output \
    --sumstats-file .. \
    --ld-region ..

In [ ]:
sos run mvenloc.ipynb enloc \
    --cwd output \
    --eqtl-pip .. \
    --gwas-pip ..

### Summary

In [ ]:
head enloc.enrich.out

In [ ]:
head enloc.sig.out

In [ ]:
head enloc.snp.out

## Command interface

In [ ]:
sos run mvenloc.ipynb -h

## Implementation

In [7]:
[global]
parameter: cwd = path
parameter: container = ""

### Step 0: data formatting

#### Extract common SNPS between the GWAS summary statistics and eQTL data 

In [8]:
[merger]
# eQTL summary statistics as a list of RData
parameter: eqtl_sumstats = path 
# GWAS summary stats in gz format
parameter: gwas_sumstats = path 
input: eqtl_sumstats, gwas_sumstats
output: f"{cwd:a}/{eqtl_sumstats:bn}.standardized.gz", f"{cwd:a}/{gwas_sumstats:bn}.standardized.gz"
R: expand = "${ }"

    ###
    # functions
    ###

    allele.qc = function(a1,a2,ref1,ref2) {
        # a1 and a2 are the first data-set
        # ref1 and ref2 are the 2nd data-set
        # Make all the alleles into upper-case, as A,T,C,G:
            a1 = toupper(a1)
            a2 = toupper(a2)
            ref1 = toupper(ref1)
            ref2 = toupper(ref2)
        # Strand flip, to change the allele representation in the 2nd data-set
        strand_flip = function(ref) {
            flip = ref
            flip[ref == "A"] = "T"
            flip[ref == "T"] = "A"
            flip[ref == "G"] = "C"
            flip[ref == "C"] = "G"
            flip
        }
        flip1 = strand_flip(ref1)
        flip2 = strand_flip(ref2)
        snp = list()
        # Remove strand ambiguous SNPs (scenario 3)
        snp[["keep"]] = !((a1=="A" & a2=="T") | (a1=="T" & a2=="A") | (a1=="C" & a2=="G") | (a1=="G" & a2=="C"))
        # Remove non-ATCG coding
        snp[["keep"]][ a1 != "A" & a1 != "T" & a1 != "G" & a1 != "C" ] = F
        snp[["keep"]][ a2 != "A" & a2 != "T" & a2 != "G" & a2 != "C" ] = F
        # as long as scenario 1 is involved, sign_flip will return TRUE
        snp[["sign_flip"]] = (a1 == ref2 & a2 == ref1) | (a1 == flip2 & a2 == flip1)
        # as long as scenario 2 is involved, strand_flip will return TRUE
        snp[["strand_flip"]] = (a1 == flip1 & a2 == flip2) | (a1 == flip2 & a2 == flip1)
        # remove other cases, eg, tri-allelic, one dataset is A C, the other is A G, for example.
        exact_match = (a1 == ref1 & a2 == ref2) 
        snp[["keep"]][!(exact_match | snp[["sign_flip"]] | snp[["strand_flip"]])] = F
        return(snp)
    }

    # Extract information from RData
    eqtl.split = function(eqtl){
      rows = length(eqtl)
      chr = vector(length = rows)
      pos = vector(length = rows)
      a1 = vector(length = rows)
      a2 = vector(length = rows)
      for (i in 1:rows){
        split1 = str_split(eqtl[i], ":")
        split2 = str_split(split1[[1]][2], "_")
        chr[i]= split1[[1]][1]
        pos[i] = split2[[1]][1]
        a1[i] = split2[[1]][2]
        a2[i] = split2[[1]][3]

      }
      eqtl.df = data.frame(eqtl,chr,pos,a1,a2)
    }

    remove.dup = function(df){
      df = df %>% arrange(PosGRCh37, -N)
      df = df[!duplicated(df$PosGRCh37),]
      return(df)
    }
  
    ###
    # Code
    ###
    
    # gene regions: 
    # 1 = ENSG00000203710
    # 2 = ENSG00000064687
    # 3 = ENSG00000203710
    
    # eqtl
    gene.name = scan(${_input[0]:r}, what='character')

    # initial filter of gwas variants that are in eqtl
    gwas = gwas_sumstats
    gwas_filter = gwas[which(gwas$id %in% var),]

    # create eqtl df
    eqtl.df = eqtl.split(eqtl$var)

    # allele flip
    f_gwas = gwas %>% filter(chr %in% eqtl.df$chr & PosGRCh37 %in% eqtl.df$pos)
    eqtl.df.f = eqtl.df %>% filter(pos %in% f_gwas$PosGRCh37)

    # check if there are duplicate pos
    length(unique(f_gwas$PosGRCh37))

    # multiple snps with same pos
    dup.pos = f_gwas %>% group_by(PosGRCh37) %>% filter(n() > 1) 

    f_gwas = remove.dup(f_gwas)

    qc = allele.qc(f_gwas$testedAllele, f_gwas$otherAllele, eqtl.df.f$a1, eqtl.df.f$a2)
    keep = as.data.frame(qc$keep)
    sign = as.data.frame(qc$sign_flip)
    strand = as.data.frame(qc$strand_flip)

    # sign flip
    f_gwas$z[qc$sign_flip] = -1 * f_gwas$z[qc$sign_flip]
    f_gwas$testedAllele[qc$sign_flip] = eqtl.df.f$a1[qc$sign_flip]
    f_gwas$otherAllele[qc$sign_flip] = eqtl.df.f$a2[qc$sign_flip]

    f_gwas$testedAllele[qc$strand_flip] = eqtl.df.f$a1[qc$strand_flip]
    f_gwas$otherAllele[qc$strand_flip] = eqtl.df.f$a2[qc$strand_flip]

    # remove ambigiuous 
    if ( sum(!qc$keep) > 0 ) {
      eqtl.df.f = eqtl.df.f[qc$keep,]
      f_gwas = f_gwas[qc$keep,]
    }

#### Extract common SNPS between the summary statistics and LD

In [9]:
[eqtl_1, gwas_1 (filter LD file and sumstat file)]
parameter: sumstat_file = path
# LD and region information: chr, start, end, LD file
paramter: ld_region = path
input: sumstat_file, for_each = 'ld_region'
output: f"{cwd:a}/{sumstat_file:bn}_{region[0]}_{region[1]}_{region[2]}.z.rds",
        f"{cwd:a}/{sumstat_file:bn}_{region[0]}_{region[1]}_{region[2]}.ld.rds"
R:
    # FIXME: need to filter both ways for sumstats and for LD
    # lds filtered
  
    eqtl_id = which(var %in% eqtl.df.f$eqtl)
    ld_f = ld[eqtl_id, eqtl_id]

    # ld missing
    miss = which(is.na(ld_f), arr.ind=TRUE)
    miss_r = unique(as.data.frame(miss)$row)
    miss_c = unique(as.data.frame(miss)$col)

    total_miss = unique(union(miss_r,miss_c))
    # FIXME: LD should not have missing data if properly processed by our pipeline
    # In the future we should throw an error when it happens

    if (length(total_miss)!=0){
    ld_f2 = ld_f[-total_miss,]
    ld_f2 = ld_f2[,-total_miss]
    dim(ld_f2)
    }else{ld_f2 = ld_f}
    
    f_gwas.f = f_gwas %>% filter(id %in% eqtl_id.f$eqtl)


### Step 1: fine-mapping

In [10]:
[eqtl_2, gwas_2 (finemapping)]
# FIXME: RDS file should have included region information
output: f"{_input[0]:nn}.susieR.rds", f"{_input[0]:nn}.susieR_plot.rds"
R:
    susie_results = susieR::susie_rss(z = f_gwas.f$z,R = ld_f2, check_prior = F)
    susieR::susie_plot(susie_results,"PIP")
    susie_results$z = f_gwas.f$z
    susieR::susie_plot(susie_results,"z_original")

### Step 2: fine-mapping results processing

#### Construct eQTL annotation file using eQTL SNP PIPs and credible sets 

In [ ]:
[eqtl_3 (create signal cluster using CS)]
output: f"{_input[0]:nn}.enloc_annot.gz"
R:
    cs = eqtl[["sets"]][["cs"]][["L1"]]
    o_id = which(var %in% eqtl_id.f$eqtl)
    pip = eqtl$pip[o_id]
    eqtl_annot = cbind(eqtl_id.f, pip) %>% mutate(gene = gene.name,cluster = -1, cluster_pip = 0, total_snps = 0)

    for(snp in cs){
      eqtl_annot$cluster[snp] = 1
      eqtl_annot$cluster_pip[snp] = eqtl[["sets"]][["coverage"]]
       eqtl_annot$total_snps[snp] = length(cs)
    }

    eqtl_annot1 = eqtl_annot %>% filter(cluster != -1)%>% 
      mutate(annot = sprintf("%s:%d@=%e[%e:%d]",gene,cluster,pip,cluster_pip,total_snps)) %>%
      select(c(chr,pos,eqtl,a1,a2,annot))
      
    
    # FIXME: repeats whole process (extracting+fine-mapping+cs creation) 3 times before this next step 
    eqtl_annot_comb = rbind(eqtl_annot3, eqtl_annot1, eqtl_annot2)

    # FIXME: write to a zip file
    write.table(eqtl_annot_comb, file = "eqtl.annot.txt", col.names = T, row.names = F, quote = F)

#### Export GWAS PIP

In [ ]:
[gwas_3 (format PIP into enloc GWAS input)]
output: f"{_input[0]:nn}.enloc_gwas.gz"
R:
    gwas_annot1 = f_gwas.f %>% mutate(pip = susie_results$pip)
    
    # FIXME: repeat whole process (extracting common snps + fine-mapping) 3 times before the next steps
    
    gwas_annot_comb = rbind(gwas_annot3, gwas_annot1, gwas_annot2)
    gwas_loc_annot = gwas_annot_comb %>% select(id, chr, PosGRCh37,z)
    write.table(gwas_loc_annot, file = "loc.gwas.txt", col.names = F, row.names = F, quote = F)

bash:    
    perl format2torus.pl loc.gwas.txt > loc2.gwas.txt
    
R:
    loc = data.table::fread("loc2.gwas.txt")
    loc = loc[["V2"]]
    
    gwas_annot_comb2 = gwas_annot_comb %>% select(id, chr, PosGRCh37,pip)
    gwas_annot_comb2 = cbind(gwas_annot_comb2, loc) %>% select(id, loc, pip)
    
    write.table(gwas_annot_comb2, file = "gwas.pip.txt", col.names = F, row.names = F, quote = F)

bash:
    perl format2torus.pl gwas.pip.txt | gzip --best > gwas.pip.gz

### Step 3: Colocalization with FastEnloc

In [ ]:
[enloc]
# eQTL summary statistics as a list of RData
# FIXME: to replace later
parameter: eqtl_pip = path 
# GWAS summary stats in gz format
parameter: gwas_pip = path 
input: eqtl_pip, gwas_pip
output: f"{cwd:a}/{eqtl_pip:bnn}.{gwas_pip:bnn}.xx.gz"
bash:
    fastenloc -eqtl eqtl.annot.txt.gz -gwas gwas.pip.txt.gz
    sort -grk6 prefix.enloc.sig.out | gzip --best > prefix.enloc.sig.sorted.gz
    rm -f prefix.enloc.sig.out